In [1]:
from langchain.schema import HumanMessage, SystemMessage
from langchain.chains import ConversationChain, HypotheticalDocumentEmbedder, LLMChain, RetrievalQA

from langchain import PromptTemplate
from langchain.memory import ConversationSummaryMemory
from langchain_cohere import CohereRerank
from langchain_community.chat_models import ChatMlflow
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.embeddings import MlflowEmbeddings
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor, EmbeddingsFilter, DocumentCompressorPipeline
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from domino_data.vectordb import domino_pinecone3x_init_params, domino_pinecone3x_index_params
from langchain_pinecone import PineconeVectorStore
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

from typing import List
import os
from pinecone import Pinecone
import sys
import logging
from mlflow.deployments import get_deploy_client

import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:317: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [ ]:
PINECONE_ENV = os.environ['PINECONE_ENV']

In [2]:
# Helper function for printing docs

def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

In [3]:
# initialize embedding
embeddings = MlflowEmbeddings(
    target_uri=os.environ["DOMINO_MLFLOW_DEPLOYMENTS"],
    endpoint="embedding-ada-002ja2"
)

In [4]:
#Domino Vector Data Source name
datasource_name = "mrag-fin-docs-ja"
# Load Domino Pinecone Data Source Configuration 
pc = Pinecone(**domino_pinecone3x_init_params(datasource_name))

# Load Pinecone Index
index_name = "mrag-fin-docs"
index = pc.Index(**domino_pinecone3x_index_params(datasource_name, index_name))
text_field = "text"  # switch back to normal index for langchain
vectorstore = PineconeVectorStore(  
    index, embeddings, text_field   # Using embedded data from Domino AI Gateway Endpoint
)

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 20})

In [ ]:
index.describe_index_stats()

In [ ]:
chatLLM = ChatMlflow(
        target_uri=os.environ["DOMINO_MLFLOW_DEPLOYMENTS"],
        endpoint="chat-gpt4-ja", 
        temperature=0.0,
    )

In [ ]:
# Set logging for the queries
logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [ ]:
# Output parser will split the LLM result into a list of queries
class LineList(BaseModel):
    # "lines" is the key (attribute name) of the parsed output
    lines: List[str] = Field(description="Lines of text")


class LineListOutputParser(PydanticOutputParser):
    def __init__(self) -> None:
        super().__init__(pydantic_object=LineList)

    def parse(self, text: str) -> LineList:
        lines = text.strip().split("\n")
        return LineList(lines=lines)

In [ ]:
output_parser = LineListOutputParser()

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant with expertise in financial analysis. Your task is to generate five 
    different versions of the given user question to retrieve relevant documents from a vector 
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search. 
    Provide these alternative questions separated by newlines.
    Original question: {question}""",
)


# Chain
llm_chain = LLMChain(llm=chatLLM, prompt=QUERY_PROMPT, output_parser=output_parser)

# Other inputs
question = "How did the Americas do in net sales in FY23?"

In [ ]:
# Run
retriever = MultiQueryRetriever(
    retriever=vectorstore.as_retriever(search_kwargs={"k": 20}), llm_chain=llm_chain, parser_key="lines"
)  # "lines" is the key (attribute name) of the parsed output

# Results
unique_docs = retriever.get_relevant_documents(
    query="How profitable is Apple?"
)
len(unique_docs)

In [ ]:

docs = retriever.get_relevant_documents( "How did the Americas do in net sales in FY23?"
    #"Were there any product annoucements by Apple in FY23?"
)
pretty_print_docs(docs)

In [ ]:
# built-in compressors: filters
compressor = LLMChainExtractor.from_llm(chatLLM)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, 
    base_retriever=vectorstore.as_retriever(search_kwargs={"k": 20})
)

compressed_docs = compression_retriever.get_relevant_documents(
    "How did the Americas do in net sales in FY23?"
)generate in FY23?

pretty_print_docs(compressed_docs)

In [ ]:
relevant_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.76)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=relevant_filter, base_retriever=retriever
)

compressed_docs = compression_retriever.get_relevant_documents("How much revenue did the Americas generate in FY23?"
   # "Was there any pending litigation?"
)
pretty_print_docs(compressed_docs)

In [ ]:
redundant_filter = EmbeddingsRedundantFilter(embeddings=embeddings)
pipeline_compressor = DocumentCompressorPipeline(
    transformers=[redundant_filter, relevant_filter]
)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline_compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.get_relevant_documents("How much revenue did the Americas generate in FY23?"
   # "Was there any pending litigation?"
)
pretty_print_docs(compressed_docs)

In [ ]:
relevant_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.76)
redundant_filter = EmbeddingsRedundantFilter(embeddings=embeddings)
pipeline_compressor = DocumentCompressorPipeline(
    transformers=[redundant_filter, relevant_filter]
)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline_compressor, base_retriever=retriever
)

In [ ]:
prompt_template = """As an advanced Retrieve-and-Generate (RAG) Chatbot with expertise in financial analysis, your task is to dissect corporate filings (e.g., 10-K, 10-Q, 8-K reports) of publicly traded companies and provide detailed, accurate responses to user queries about the company's financial health, market position, and future prospects. When interacting, adhere to the following guidelines:
- Context Understanding: Thoroughly comprehend the context provided, which includes excerpts or summaries from the company’s latest corporate filings. This context is your foundational source for analysis.
- Question Analysis: Analyze the user’s specific question to understand which aspect of the corporate filings it relates to, such as financial performance, risk factors, management discussion, market position, future outlook, or investment considerations.
- Structured Response: Base your response on the appropriate section(s) of the corporate filings pertinent to the question, ensuring your answer is data-driven.
- Detailed Inquiry Response: Address financial performance, risk factors, management discussion, market position, future outlook, or investment considerations with focused, evidence-backed answers.
- Evidence-Based Justification: Support your responses with direct evidence from the provided context, offering insights derived from the corporate filings.
- Clarity and Precision: Maintain clarity and precision in your responses, using accessible language and avoiding or explaining necessary financial jargon.
- Handling Unknown Answers: If the information needed to answer the question is not available in the provided context or exceeds the chatbot's analysis capabilities, respond with, "I don't have enough information to answer that question accurately. Could you provide more details or ask about another aspect?"
- Addressing Irrelevant Questions: If the question is not related to the context of corporate filings, politely respond with, "I'm here to help analyze financial documents and related inquiries. Could you ask a question related to the company's corporate filings?"
- Primary Objective is to deliver insightful, accurate, and helpful answers that enable users to make informed decisions based on corporate filings analysis. Each response should be tailored to the user's question, enhancing understanding of the company's financial status and strategic direction.
You are given the following question and extracted parts as context. 
Question: {question}
=========
{context}
=========
Answer in Markdown:
"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["question", "context"])
#
chain_type_kwargs = {"prompt": PROMPT}

In [ ]:
#You are an AI assistant with expertise in financial analysis, your task is to dissect corporate filings (e.g., 10-K, 10-Q, 8-K reports) of publicly traded companies and provide detailed, accurate responses to the following user question about the company's financial health, market position, and future prospects. 
#. When interacting, adhere to the following guidelines:
prompt_template = """You are an AI assistant with expertise in financial analysis. You are given the following extracted parts and a question. 
Question: {question}
=========
{context}
=========
Answer in Markdown:
"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["question", "context"])
#
chain_type_kwargs = {"prompt": PROMPT}

In [ ]:
qa_chain = RetrievalQA.from_chain_type(llm=chatLLM,
                                       chain_type="stuff",
                                       chain_type_kwargs={"prompt": PROMPT},
                                       retriever=compression_retriever, #vectorstore.as_retriever(search_kwargs={"k": 5}),
                                       return_source_documents=True
                                      )

In [ ]:
user_question = input("Please ask your question:")
result = qa_chain(user_question)

In [ ]:
result["result"]

In [ ]:
result["result"]

In [ ]:
result["result"]

In [ ]:
result["result"]

In [ ]:
print(result)

In [ ]:
user_question = input("Please ask your financial question:")
compressed_docs = compression_retriever.get_relevant_documents(user_question)
# Print the relevant documents from using the embeddings and reranker
print(compressed_docs)

In [ ]:
# Setup HyDE

hyde_prompt_template = """As an advanced Retrieve-and-Generate (RAG) Chatbot with expertise in financial analysis, your task is to dissect corporate filings (e.g., 10-K, 10-Q, 8-K reports) of publicly traded companies and provide detailed, accurate responses to the following user question about the company's financial health, market position, and future prospects. When interacting, adhere to the following guidelines:
Your primary objective is to deliver insightful, accurate, and helpful answers that enable users to make informed decisions based on corporate filings analysis. Each response should be tailored to the user's question, enhancing understanding of the company's financial status and strategic direction. 
"Please answer the user's question below \n 
Question: {question}
Answer:
"""

hyde_prompt = PromptTemplate(input_variables=["question"], template=hyde_prompt_template)
hyde_llm_chain = LLMChain(llm=chatLLM, prompt=hyde_prompt)

hyde_embeddings = HypotheticalDocumentEmbedder(
    llm_chain=hyde_llm_chain, base_embeddings=embeddings
)

In [ ]:
# Get relevant docs through vector DB

SIMILARITY_THRESHOLD = 0.5

# Number of texts to match (may be less if no suitable match)
NUM_TEXT_MATCHES = 5

# Number of texts to return from reranking
NUM_RERANKING_MATCHES = 3

# Create prompt
template = """As an advanced Retrieve-and-Generate (RAG) Chatbot with expertise in financial analysis, your task is to dissect corporate filings (e.g., 10-K, 10-Q, 8-K reports) of publicly traded companies and provide detailed, accurate responses to the following user question about the company's financial health, market position, and future prospects. When interacting, adhere to the following guidelines:If the question is not related to the context of corporate filings, politely respond with, 'Hi, I'm here to help analyze financial documents and related inquiries. Could you ask a question related to the company's corporate filings?'
Your primary objective is to deliver insightful, accurate, and helpful answers that enable users to make informed decisions based on corporate filings analysis. Each response should be tailored to the user's question, enhancing understanding of the company's financial status and strategic direction. pertaining to policy coverage.
Here is some relevant context: {context}
"""

# Load the reranking model
colbert = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

# Get relevant docs through vector DB
def get_relevant_docs(user_input, num_matches=NUM_TEXT_MATCHES, use_hyde=False):
   
    if use_hyde:
        embedded_query = hyde_embeddings.embed_query(user_input)
    else:
        embedded_query = embeddings.embed_query(user_input)
        
 
    relevant_docs = index.query(
        vector=embedded_query,
        top_k=num_matches,
        include_values=True,
        include_metadata=True
    )

    matches = relevant_docs["matches"]
    filtered_matches = [match for match in matches if match['score'] >= SIMILARITY_THRESHOLD]
    relevant_docs["matches"] = filtered_matches

    return relevant_docs

 
def build_system_prompt(user_input, rerank=False, use_hyde=False):
    print(user_input)
    try:
        relevant_docs = get_relevant_docs(user_input)
    except Exception as e:
        print(f"Failed to get relevant documents: {e}")
        return "", "Failed to get relevant documents"

    actual_num_matches = len(relevant_docs["matches"])
    if actual_num_matches == 0:
        print("No matches found in relevant documents.")
        return "", "No matches found in relevant documents"
    
    contexts = [relevant_docs["matches"][i]["metadata"]["text"] for i in range(actual_num_matches)]
    print("num_matches: ", actual_num_matches)
    if rerank and actual_num_matches >= NUM_RERANKING_MATCHES:
        try:
            docs = colbert.rerank(query=user_input, documents=contexts, k=NUM_RERANKING_MATCHES)
        except Exception as e:
            print(f"Failed to rerank documents: {e}")
            return "", "Failed to rerank documents"
        
        try:
            result_indices = [docs[i]["result_index"] for i in range(NUM_RERANKING_MATCHES)]
        except (IndexError, KeyError) as e:
                print(f"Invalid result indices: {e}")
                return "", "Invalid result indices"
        try:    
            contexts = [contexts[index] for index in result_indices]
        except IndexError as e:
            print(f"Indexing error: {e}")
            return "", "Indexing error"
    
    # Create prompt
    template = """As an advanced Retrieve-and-Generate (RAG) Chatbot with expertise in financial analysis, your task is to dissect corporate filings (e.g., 10-K, 10-Q, 8-K reports) of publicly traded companies and provide detailed, accurate responses to the following user question about the company's financial health, market position, and future prospects. When interacting, adhere to the following guidelines
Your primary objective is to deliver insightful, accurate, and helpful answers that enable users to make informed decisions based on corporate filings analysis. Each response should be tailored to the user's question, enhancing understanding of the company's financial status and strategic direction.
Here is some relevant context: {context}"""
    
    prompt_template = PromptTemplate(
        input_variables=["context"],
        template=template
    )
    
    system_prompt = prompt_template.format(context=contexts)
    print(contexts)
    return system_prompt, contexts

# Query the Open AI Model
def queryAIModel(user_input, llm_name="openai", use_hyde=False):

    if not user_input:
        return "Please provide an input"
    
    system_prompt = build_system_prompt(user_input)
   # print(system_prompt)
    messages = [
        SystemMessage(
            content=system_prompt[0]
        ),
        HumanMessage(
            content=user_input
        ),
    ]
    
    output = conversationChat.predict(input=messages)

    return output

In [ ]:
# Ask a question ; uncomment to test
user_question = input("Please provide your question here :")
result = queryAIModel(user_question)


In [ ]:
print(result)

In [ ]:
index.describe_index_stats()


In [ ]:
%run /mnt/code/services/data_ingestion.py
